In [2]:
import json

# 假设文件路径分别为
path_A = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\temp.json"  # 请替换为实际路径
path_B = r"C:\Users\12860\Desktop\科研\crosscodeeval\python\modified_line_completion_split_copy.json"  # 请替换为实际路径

# 读取文件A
with open(path_A, 'r', encoding='utf-8') as f:
    data_A = json.load(f)

# 读取文件B
with open(path_B, 'r', encoding='utf-8') as f:
    data_B = json.load(f)

# 验证读取结果（可选）
print(f"文件A包含 {len(data_A)} 条记录")
print(f"文件B包含 {len(data_B)} 条记录")

文件A包含 242 条记录
文件B包含 2665 条记录


In [4]:
import re

# 确保A和B的记录数量相同
if len(data_A) != len(data_B):
    print(f"警告：data_A({len(data_A)}) 和 data_B({len(data_B)}) 的记录数量不同")

# 平行遍历A和B的所有记录
for idx, (item_a, item_b) in enumerate(zip(data_A, data_B)):
    print(f"\n--- 第 {idx+1} 条记录 ---")
    
    # 遍历每条记录的所有字段（假设A和B字段名完全一致）
    for field_name in item_a.keys():
        # 获取A和B中对应字段的值
        value_a = item_a[field_name]
        value_b = item_b[field_name]
        
        # 确保两个字段都是字符串
        if not (isinstance(value_a, str) and isinstance(value_b, str)):
            print(f"字段 '{field_name}' 不是字符串类型，跳过处理")
            continue
        
        # 在A的字段中查找所有<LibFunc->标记
        pattern = r'<LibFunc->.*?>'
        matches = re.findall(pattern, value_a)
        
        if matches:
            print(f"字段 '{field_name}' 中找到 {len(matches)} 个标记:")
            for match in matches:
                # 提取A中标记后的5个字符
                match_index = value_a.index(match)
                end_index = match_index + len(match)
                target_chars = value_a[end_index:end_index+5]
                
                if not target_chars:
                    print(f"  标记 {match} 后没有足够字符")
                    continue
                
                # 在B的对应字段中查找这5个字符
                if target_chars in value_b:
                    # 找到所有匹配位置
                    positions = []
                    start = 0
                    while True:
                        pos = value_b.find(target_chars, start)
                        if pos == -1:
                            break
                        positions.append(pos)
                        start = pos + 1
                    
                    print(f"  标记 {match} 提取的字符 '{target_chars}' 在B中找到，位置: {positions}")
                else:
                    print(f"  标记 {match} 提取的字符 '{target_chars}' 在B中未找到")
        else:
            print(f"字段 '{field_name}' 中没有找到<LibFunc->标记")

警告：data_A(242) 和 data_B(2665) 的记录数量不同

--- 第 1 条记录 ---
字段 'completion' 中没有找到<LibFunc->标记
字段 'prefix_split_1' 中没有找到<LibFunc->标记
字段 'prefix_split_2' 中找到 2 个标记:
  标记 <LibFunc->(use tokenizer to encode the text)> 提取的字符 'token' 在B中找到，位置: [31, 61, 88, 201, 300, 340, 398, 531, 580, 624, 654, 711, 810, 842, 913, 930, 985]
  标记 <LibFunc->(use tokenizer to decode prompt_ids)> 提取的字符 'token' 在B中找到，位置: [31, 61, 88, 201, 300, 340, 398, 531, 580, 624, 654, 711, 810, 842, 913, 930, 985]
字段 'prefix_split_3' 中找到 3 个标记:
  标记 <LibFunc->(use tokenizer to get number of tokens)> 提取的字符 'get_n' 在B中找到，位置: [222]
  标记 <LibFunc->(use generator to generate)> 提取的字符 'gener' 在B中找到，位置: [251, 298, 329, 342, 352, 437, 900]
  标记 <LibFunc->(use tokenizer to decode...)> 提取的字符 'token' 在B中找到，位置: [37, 230, 448, 488, 546, 662, 800, 859, 883]
字段 'suffix_split_1' 中找到 3 个标记:
  标记 <LibFunc->(use generator to generate a single token)> 提取的字符 'gener' 在B中找到，位置: [33, 46, 71, 399]
  标记 <LibFunc->(use tokenizer to decode the nearest max_s

In [6]:
import re
import json
import os

# 确保A和B的记录数量相同
if len(data_A) != len(data_B):
    print(f"警告：data_A({len(data_A)}) 和 data_B({len(data_B)}) 的记录数量不同")

# 用于存储处理后的结果
processed_data = []

# 平行遍历A和B的所有记录
for idx, (item_a, item_b) in enumerate(zip(data_A, data_B)):
    processed_item = {}  # 处理后的当前记录
    
    # 遍历每条记录的所有字段
    for field_name in item_a.keys():
        value_a = item_a[field_name]
        value_b = item_b[field_name]
        
        # 确保两个字段都是字符串
        if not (isinstance(value_a, str) and isinstance(value_b, str)):
            print(f"字段 '{field_name}' 不是字符串类型，直接使用B的值")
            processed_item[field_name] = value_b
            continue
        
        # 在A的字段中查找所有<LibFunc->标记
        pattern = r'<LibFunc->.*?>'
        matches = re.findall(pattern, value_a)
        
        if not matches:
            # 没有找到标记，使用B的字符串
            print(f"字段 '{field_name}' 中没有找到<LibFunc->标记，使用B的值")
            processed_item[field_name] = value_b
            continue
        
        # 处理第一个标记（用于前面部分替换）
        first_match = matches[0]
        first_match_index = value_a.index(first_match)
        first_end_index = first_match_index + len(first_match)
        first_target = value_a[first_end_index:first_end_index+5]
        
        # 处理最后一个标记（用于后面部分替换）
        last_match = matches[-1]
        last_match_index = value_a.rindex(last_match)  # 使用rindex确保找到最后一个
        last_end_index = last_match_index + len(last_match)
        last_target = value_a[last_end_index:last_end_index+5]
        
        # 检查最后一个标记提取是否成功，失败则尝试前一个
        current_last_idx = len(matches) - 1
        while not last_target and current_last_idx > 0:
            current_last_idx -= 1
            last_match = matches[current_last_idx]
            last_match_index = value_a.rindex(last_match)
            last_end_index = last_match_index + len(last_match)
            last_target = value_a[last_end_index:last_end_index+5]
        
        if not first_target or not last_target:
            # 提取失败，使用B的字符串
            print(f"字段 '{field_name}' 标记提取字符失败，使用B的值")
            processed_item[field_name] = value_b
            continue
        
        # 在B中查找第一个目标字符的首次出现（前面部分）
        b_first_pos = value_b.find(first_target)
        # 在B中查找最后一个目标字符的末次出现（后面部分）
        b_last_pos = value_b.rfind(last_target)
        
        if b_first_pos == -1 or b_last_pos == -1:
            # 未找到匹配，使用B的字符串
            print(f"字段 '{field_name}' 提取的字符在B中未找到，使用B的值")
            processed_item[field_name] = value_b
            continue
        
        # 执行替换操作
        # 1. 处理前面部分：B中首次匹配前的部分 + A中第一个标记及之后部分
        a_part_from_first = value_a[first_match_index:]
        processed_part1 = value_b[:b_first_pos] + a_part_from_first
        
        # 2. 处理后面部分：从前面处理结果中截取到最后一个标记前 + B中末次匹配及之后部分
        # 找到最后一个标记在中间结果中的位置
        final_last_index = processed_part1.rindex(last_match)
        final_last_end = final_last_index + len(last_match)
        processed_value = processed_part1[:final_last_end] + value_b[b_last_pos:]
        
        print(f"字段 '{field_name}' 处理成功")
        processed_item[field_name] = processed_value
    
    # 将处理后的记录添加到结果列表
    processed_data.append(processed_item)

# 在同目录下创建并保存temp_postProcessed.json文件
output_path = os.path.join(os.path.dirname(path_A), "temp_postProcessed.json")
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=2)

print(f"\n处理完成，结果已保存到: {output_path}")
print(f"处理后的数据包含 {len(processed_data)} 条记录")
    

警告：data_A(242) 和 data_B(2665) 的记录数量不同
字段 'completion' 中没有找到<LibFunc->标记，使用B的值
字段 'prefix_split_1' 中没有找到<LibFunc->标记，使用B的值
字段 'prefix_split_2' 处理成功
字段 'prefix_split_3' 处理成功
字段 'suffix_split_1' 处理成功
字段 'suffix_split_2' 处理成功
字段 'suffix_split_3' 处理成功
字段 'suffix_split_4' 处理成功
字段 'suffix_split_5' 处理成功
字段 'suffix_split_6' 处理成功
字段 'suffix_split_7' 处理成功
字段 'completion' 中没有找到<LibFunc->标记，使用B的值
字段 'prefix_split_1' 处理成功
字段 'prefix_split_2' 处理成功
字段 'prefix_split_3' 处理成功
字段 'prefix_split_4' 处理成功
字段 'suffix_split_1' 处理成功
字段 'completion' 中没有找到<LibFunc->标记，使用B的值
字段 'prefix_split_1' 处理成功
字段 'prefix_split_2' 处理成功
字段 'suffix_split_1' 处理成功
字段 'suffix_split_2' 处理成功
字段 'completion' 中没有找到<LibFunc->标记，使用B的值
字段 'prefix_split_1' 处理成功
字段 'prefix_split_2' 处理成功
字段 'suffix_split_1' 处理成功
字段 'suffix_split_2' 处理成功
字段 'suffix_split_3' 处理成功
字段 'suffix_split_4' 处理成功
字段 'suffix_split_5' 处理成功
字段 'suffix_split_6' 处理成功
字段 'suffix_split_7' 处理成功
字段 'suffix_split_8' 处理成功
字段 'completion' 中没有找到<LibFunc->标记，使用B的值
字段 'prefix_split_1

In [7]:
import json
import os
import re

# 读取temp_postProcessed.json文件
input_path = os.path.join(os.path.dirname(path_A), "temp_postProcessed.json")
with open(input_path, 'r', encoding='utf-8') as f:
    processed_data = json.load(f)

# 用于存储合并后的结果
completed_data = []

# 处理每个字典项
for item in processed_data:
    merged_item = {}
    
    # 保留completion字段
    if "completion" in item:
        merged_item["completion"] = item["completion"]
    
    # 收集所有prefix_split和suffix_split
    prefix_splits = {}
    suffix_splits = {}
    
    # 正则表达式匹配prefix_split_[n]和suffix_split_[n]格式的键
    prefix_pattern = r'^prefix_split_(\d+)$'
    suffix_pattern = r'^suffix_split_(\d+)$'
    
    for key, value in item.items():
        # 匹配prefix_split
        prefix_match = re.match(prefix_pattern, key)
        if prefix_match:
            n = int(prefix_match.group(1))
            prefix_splits[n] = value
        
        # 匹配suffix_split
        suffix_match = re.match(suffix_pattern, key)
        if suffix_match:
            n = int(suffix_match.group(1))
            suffix_splits[n] = value
    
    # 按n的大小排序并合并prefix_split
    if prefix_splits:
        sorted_prefix_nums = sorted(prefix_splits.keys())
        merged_prefix = ''.join([prefix_splits[n] for n in sorted_prefix_nums])
        merged_item["merged_prefix"] = merged_prefix
    
    # 按n的大小排序并合并suffix_split
    if suffix_splits:
        sorted_suffix_nums = sorted(suffix_splits.keys())
        merged_suffix = ''.join([suffix_splits[n] for n in sorted_suffix_nums])
        merged_item["merged_suffix"] = merged_suffix
    
    completed_data.append(merged_item)

# 保存合并结果到temp_completed.json
output_path = os.path.join(os.path.dirname(path_A), "temp_completed.json")
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(completed_data, f, ensure_ascii=False, indent=2)

print(f"合并完成，结果已保存到: {output_path}")
print(f"合并后的数据包含 {len(completed_data)} 条记录")
    

合并完成，结果已保存到: C:\Users\12860\Desktop\科研\crosscodeeval\python\temp_completed.json
合并后的数据包含 242 条记录
